# PROYECTO 1

**Curso:** Operaciones de Aprendizaje de Máquina

**Estudiantes:**
- Juan José García
- Ruben Dario Hoyos
- José Rafael Peña

## Setup

### Importación de librerías

In [1]:
# General modules
from pathlib import Path
import os
import requests
from typing import List
from dataclasses import dataclass
import pandas as pd
import pprint as pp

In [2]:
# Sklearn modules
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split

In [3]:
# Tensorflow module
import tensorflow as tf

# TFX components
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.v1.components import ImportSchemaGen
from tfx.components import Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

# TFDV modules
import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import schema_pb2

2025-03-05 18:02:24.097723: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 18:02:24.103364: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 18:02:24.121346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-05 18:02:24.156779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-05 18:02:24.156888: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 18:02:24.203139: I tensorflow/core/platform/cpu_feature_guard.cc:

In [4]:
import ml_metadata as mlmd
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

### Definición de carpetas

In [5]:
# Directory of the raw data files
data_root = Path('./data/covertype')

# Directory of the preprocessed data files
data_root_prepro = Path('./data/covertype_prepro')

# Path to the raw training data
data_filepath = data_root / 'covertype_train.csv'

# Ensure the data_root directory exists
data_root.mkdir(parents=True, exist_ok=True)

# Ensure the data_root_prepro directory exists
data_root_prepro.mkdir(parents=True, exist_ok=True)

# Directory of the pipeline metadata store
pipeline_root = Path('./pipeline/')

# Ensure the pipeline_root directory exists
pipeline_root.mkdir(parents=True, exist_ok=True)

### Carga de datos

In [6]:
# Download data if it doesn't exist
if not data_filepath.is_file():
    # URL for the dataset
    # https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export=download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    
    r = requests.get(url, allow_redirects=True, stream=True)
    
    data_filepath.write_bytes(r.content)

## Pasos proyecto

### **2.1** Carga el dataset

In [7]:
df = pd.read_csv(data_filepath)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

### **3** Selección de características

In [9]:
@dataclass
class DataConfig:
    target_col: str
    non_numeric_cols: List[str]
    final_df_path: Path

# Creating an instance with specific values
config = DataConfig(
    target_col="Cover_Type",
    non_numeric_cols=list(df.select_dtypes(include=['object']).columns),
    final_df_path= data_root_prepro / "covertype_preprocessed.csv"
)

La ejecución de la siguiente celda se omite mediante el comando `%%script false --no-raise-error`, ya que contiene la normalización de los datos, un proceso que ya se realizó previamente según lo indicado en el documento:  

> **"Recuerde que, primero, debe preparar las características de entrada y de destino:"**  

Sin embargo, más adelante en el documento se asume que los datos conservan sus valores originales, por lo que la normalización se aplica posteriormente utilizando las herramientas de TFX.

In [10]:
%%script false --no-raise-error
# Drop non-numeric columns
df_1 = df.drop(columns=config.non_numeric_cols)

# Separate features and label
X = df_1.drop(columns=[config.target_col])
y = df_1[config.target_col].astype('category')

# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame with original column names
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Implement f_classif as score function and select the 8 best columns
selector = SelectKBest(score_func=f_classif, k=8)
selector.fit(X, y)

# Create and print a df comparing the column and the result (if its retained or not)
selected_columns_df = pd.DataFrame({
    'Column': X_scaled.columns,
    'Retain': selector.get_support()
})
selected_columns_df

In [11]:
# Drop non-numeric columns
df_1 = df.drop(columns=config.non_numeric_cols)

# Separate features and label
X = df_1.drop(columns=[config.target_col])
y = df_1[config.target_col].astype('category')

# Implement f_classif as score function and select the 8 best columns
selector = SelectKBest(score_func=f_classif, k=8)
selector.fit(X, y)

# Select the best features using boolean mask
X_selected = X.loc[:, selector.get_support()]

# Create and print a df comparing the column and the result (if its retained or not)
selected_columns_df = pd.DataFrame({
    'Column': X.columns,
    'Retain': selector.get_support()
})
selected_columns_df

,Column,Retain
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,True
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


In [12]:
# Add the target column back
final_df = X_selected.copy()
final_df[config.target_col] = y.values

# Save the updated dataframe to CSV
final_df.to_csv(config.final_df_path, index=False)

<span style="color:red; font-weight:bold;">NOTA:</span> Se debe tener cargado el dataset (`final_df`) en memoria, pues posteriormente se debe hacer una división para algunas pruebas.


### **4.1** Configurar el contexto interactivo

In [13]:
context = InteractiveContext(pipeline_root=str(pipeline_root))

### **4.2** Generando ejemplos

In [14]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=str(data_root_prepro))

# Execute the component
context.run(example_gen)

print("CsvExampleGen ok")

CsvExampleGen ok


### **4.3** Estadísticas

In [15]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: pipeline/CsvExampleGen/examples/1


In [16]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

print('StatisticsGen OK')

StatisticsGen OK


In [17]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

Se puede observar que, efectivamente, como se menciona en el documento, la columna `cover_type` tiene resaltado en rojo el porcentaje de valores en 0. Esto no es preocupante, ya que es la *target* del problema y una variable categórica.  

Por otro lado, se puede mencionar que la *feature* con el coeficiente de variación más alto (`avg/std`) es `Vertical_Distance_To_Hydrology`.

### **4.4** Inferir Esquema

In [18]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

print('SchemaGen OK')

SchemaGen OK


In [19]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


### **4.5** Curando Esquema

In [20]:
# Load schema as tensorflow_metadata.proto.v0.schema_pb2
schema_path = schema_gen.outputs['schema'].get()[0].uri + "/schema.pbtxt"
schema = tfdv.load_schema_text(schema_path)
type(schema)

tensorflow_metadata.proto.v0.schema_pb2.Schema

In [21]:
# Set domains for Hillshade_9am, Hillshade_Noon, Slope, Cover_Type
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(min=0, max=99))
domain_cover_type = schema_pb2.StringDomain(value=['0','1', '2', '3', '4', '5', '6'], is_categorical=True)
tfdv.set_domain(schema, 'Cover_Type', domain_cover_type)

Se observó que a pesar de agregar un dominio tipo String y especificar que es categorico, el tipo de la variable `Cover_Type` no cambió por lo que manualmente se asigna el tipo de dato correcto y que al mostrar el esquema, el tipo aparezca como `STRING`

In [22]:
# Manually assign Cover_Type feature type to FeatureType.BYTES
schema.feature[0].type = schema_pb2.FeatureType.BYTES

En los siguientes bloques se muestra que el esquema ha cambiado solo en memoria. Sin embargo, aún es necesario guardarlo en la metadata de SchemaGen para que los cambios sean persistentes y reconocidos por el pipeline.

In [23]:
# Display the updated schema with domains
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 99
'Vertical_Distance_To_Hydrology',INT,required,,-


,Values
Domain,
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


In [24]:
# Display the old schema with no domains
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


La siguiente celda garantiza que este quede guardado y ahora sí se podrán observar los cambios por medio del artefacto ya creado schema_gen

In [25]:
# Overwrite the file
tfdv.write_schema_text(schema, schema_path)

# Display updated schema with SchemaGen
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 99
'Vertical_Distance_To_Hydrology',INT,required,,-


,Values
Domain,
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


### **4.6** Entornos de esquema

In [26]:
def generate_serving_csv(df: pd.DataFrame,  path: Path, target: str = config.target_col) -> Path:
    train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
    test_df = test_df.drop(columns=[target])
    test_df.to_csv(path, index=False)
    return str(path)

serving_data = generate_serving_csv(final_df, data_root / "serving_data.csv")

In [27]:
options = tfdv.StatsOptions(schema=schema)
serving_stats = tfdv.generate_statistics_from_csv(serving_data, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [28]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('training')
schema.default_environment.append('serving')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, config.target_col).not_in_environment.append('serving')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='serving')

tfdv.display_anomalies(serving_anomalies_with_env)

Se procede a guardar el archivo de schema ***tanto en la carpeta de SchemaGen para persitirlo en el pipeline y también se guarda en otro archivo local como se indica en el ejercicio***

In [29]:
# Overwrite the file in SchemaGen component files and save it in a local file
tfdv.write_schema_text(schema, schema_path)
tfdv.write_schema_text(schema, data_root.parent / "schema_entornos.pbtxt")

Como se especifica en el documento se procede a verificar que este schema sí tenga los cambios realizados.

> **Como verificación, debe mostrar el esquema que acaba de guardar y verificar que contiene los cambios introducidos**

La opción `display_schema` no brinda ninguna información respecto a los entornos, pero mostrando el archivo texto se debe observar los entornos como se presenta en la documentación:
```
default_environment: "TRAINING"
default_environment: "SERVING"
```

In [30]:
print(schema.default_environment)

['training', 'serving']


In [31]:
print(schema)

feature {
  name: "Cover_Type"
  type: BYTES
  string_domain {
    value: "0"
    value: "1"
    value: "2"
    value: "3"
    value: "4"
    value: "5"
    value: "6"
    is_categorical: CATEGORICAL_YES
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  not_in_environment: "serving"
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Elevation"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_9am"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_Noon"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_Points"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape 

### **4.7** Nuevas estadísticas usando el esquema actualizado

In [32]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
path_local_schema = data_root.parent / "schema_entornos.pbtxt"
schema_gen_2 = ImportSchemaGen(schema_file=str(path_local_schema))

# Run the component
context.run(schema_gen_2)

print('ImportSchemaGen OK')
context.show(schema_gen_2.outputs['schema'])

ImportSchemaGen OK


,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',STRING,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 99
'Vertical_Distance_To_Hydrology',INT,required,,-


,Values
Domain,
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


In [33]:
compute_eval_stats = StatisticsGen(
      examples=example_gen.outputs['examples'],
      schema=schema_gen_2.outputs['schema'],
    
      )

# Execute the component
context.run(compute_eval_stats)

print('New StatisticsGen OK')

New StatisticsGen OK


In [34]:
# Show the output statistics
context.show(compute_eval_stats.outputs['statistics'])

Se observa un despliegue de estadísticas muy similar al obtenido con el esquema inferido. Esto puede deberse a que CsvExampleGen ya infiere y asigna tipos a las variables, por lo que sería necesario definir estas restricciones de tipo al momento de la lectura.

### **4.8** Comprobar anomalías

In [35]:
# Instantiate ExampleValidator with the StatisticsGen and ImportSchemaGen ingested data
example_validator = ExampleValidator(
    statistics=compute_eval_stats.outputs['statistics'],
    schema=schema_gen_2.outputs['schema'])

# Run the component.
context.run(example_validator)

print('ExampleValidator OK')

ExampleValidator OK


In [36]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Unexpected data type,Expected data of type: BYTES but got INT


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Unexpected data type,Expected data of type: BYTES but got INT


En este caso, era previsibile que resalte anomalías, ya que con las nuevas estadísticas no se logra observar el cambio en el tipo de la variable. Se intuye, sin certeza, que esto sucede porque CsvExampleGen ya infiere los tipos, y para que las estadísticas reflejen los nuevos tipos, se debe modificar la lectura.

### **4.9** Ingeniería de características

In [37]:
# Set the constants module filename
constants_module_file = 'constants.py'

In [38]:
%%writefile {constants_module_file}
# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['Elevation', 'Hillshade_9am', 'Hillshade_Noon', 'Horizontal_Distance_To_Fire_Points', 'Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Vertical_Distance_To_Hydrology']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['Slope']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'Slope': 4}

# Feature that the model will predict
LABEL_KEY = 'Cover_Type'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing constants.py


In [39]:
# Set the transform module filename
transform_module_file = 'transform.py'

In [40]:
%%writefile {transform_module_file}
import tensorflow as tf
import tensorflow_transform as tft
import constants

# Desempaquetar los contenidos del módulo de constantes
NUMERIC_FEATURE_KEYS = constants.NUMERIC_FEATURE_KEYS
BUCKET_FEATURE_KEYS = constants.BUCKET_FEATURE_KEYS
FEATURE_BUCKET_COUNT = constants.FEATURE_BUCKET_COUNT
LABEL_KEY = constants.LABEL_KEY
transformed_name = constants.transformed_name


def preprocessing_fn(inputs):
    """Función de callback de tf.transform para preprocesar entradas.
    Args:
        inputs: diccionario de características sin transformar.
    Returns:
        Diccionario de características transformadas.
    """
    outputs = {}

    # Escalar características numéricas al rango [0,1]
    for key in NUMERIC_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.scale_to_0_1(inputs[key])
    
    # Crear agrupaciones por rangos en características categorizables
    for key in BUCKET_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.bucketize(inputs[key], FEATURE_BUCKET_COUNT[key])

    return outputs


Writing transform.py


In [41]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen_2.outputs['schema'],
    module_file=transform_module_file)

# Run the component
context.run(transform)

running bdist_wheel
running build
running build_py
creating build/lib
copying transform.py -> build/lib
copying constants.py -> build/lib
installing to /tmp/tmpkfejj93x
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmpkfejj93x/.
copying build/lib/constants.py -> /tmp/tmpkfejj93x/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpkfejj93x/./tfx_user_code_Transform-0.0+09a0eef4782cbe8f7aba2ee26bae3f28d68fcf8549d54abfe60ed4cb5d0a7ed2-py3.10.egg-info
running install_scri

/home/estudiante/.cache/uv/archive-v0/D1KqFIVK3yu0lywV9UqGe/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+09a0eef4782cbe8f7aba2ee26bae3f28d68fcf8549d54abfe60ed4cb5d0a7ed2-py3-none-any.whl
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+09a0eef4782cbe8f7aba2ee26bae3f28d68fcf8549d54abfe60ed4cb5d0a7ed2-py3-none-any.whl
Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+09a0eef4782cbe8f7aba2ee26bae3f28d68fcf8549d54abfe60ed4cb5d0a7ed2-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 7
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Ahora se procede a obtener ejemplos como se indica en las instrucciones del taller.

In [42]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [43]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

Se puede observar al ejecutar la siguiente celda que:
- Hay nuevas columnas con "_xf" al final
- Los valores están 0 y 1
- Slope se encuentra entre los valores de 0 a 3, de las tres buckets que definimos

In [44]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 5)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Elevation_xf': {'floatList': {'value': [0.5686275]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.9173228]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9212598]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.21902902]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.04886944]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.14263631]}},
                           'Slope_xf': {'int64List': {'value': ['0']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.2214854]}}}}},
 {'features': {'feature': {'Elevation_xf': {'floatList': {'value': [0.5108095]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.7559055]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.79527

2025-03-05 18:04:30.346833: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### **5.1** Acceso a artefactos almacenados

Para crear un objeto `MetadataStore`, es necesario contar con una configuración de conexión. Existen diferentes formas de establecer esta configuración. Por ejemplo, se puede utilizar una base de datos en memoria mediante SQLite para experimentación rápida y ejecuciones locales:

```python
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.fake_database.SetInParent()  # Configura una base de datos ficticia en memoria.
store = metadata_store.MetadataStore(connection_config)
```

Otra opción es establecer la conexión a través de un archivo SQLite:
```python
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'ruta_a_archivo.sqlite'
connection_config.sqlite.connection_mode = 3  # READWRITE_OPENCREATE
store = metadata_store.MetadataStore(connection_config)
```

En nuestro caso, dado que contamos con un archivo de metadatos en formato SQLite, podemos utilizar la segunda opción. Sin embargo, el contexto interactivo `InteractiveContext` que hemos creado ya incluye esta configuración de conexión, lo que nos permite inicializar `MetadataStore` sin necesidad de definirla manualmente. Esto se puede verificar utilizando la función `dir`, donde se observa que el contexto contiene un atributo `metadata_connection_config`.

In [45]:
# Show attributes of the instance context from InteractiveContext
dir(context)

['_DEFAULT_SQLITE_FILENAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'beam_pipeline_args',
 'export_to_pipeline',
 'metadata_connection_config',
 'pipeline_name',
 'pipeline_root',
 'run',
 'show']

In [46]:
store = metadata_store.MetadataStore(context.metadata_connection_config)
print(f"Hay un total de {len(store.get_artifacts())} artefactos")
print("Se inspecciona el artefacto 2 de la lista (posición 1 en la lista): ")
store.get_artifacts()[1] # Este método permite ver de manera más exhaustiva todos los artefactos

Hay un total de 14 artefactos
Se inspecciona el artefacto 2 de la lista (posición 1 en la lista): 


id: 2
type_id: 16
uri: "pipeline/StatisticsGen/statistics/2"
properties {
  key: "span"
  value {
    int_value: 0
  }
}
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "statistics:2025-03-05T18:03:08.540489"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "StatisticsGen"
  }
}
custom_properties {
  key: "sample_rate_by_split"
  value {
    struct_value {
      fields {
        key: "__value__"
        value {
          string_value: "{\"eval\": 1.0, \"train\": 1.0}"
        }
      }
    }
  }
}
custom_properties {
  key: "stats_dashboard_link"
  value {
    string_value: ""
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.16.0"
  }
}
state: LIVE
name: "statistics:2025-03-05T18:03:08.540489"
type: "ExampleStatistics"
create_time_since_epoch: 1741215788603
last_update_time_since_epoch: 1741215802235

A continuación se despliegan solo los nombres de los tipos de artefactos en nuestro pipeline

In [47]:
for i in store.get_artifact_types():
    print(i.name)

Examples
ExampleStatistics
Schema
ExampleAnomalies
TransformGraph
TransformCache


Relación por medio de intuición:
- Examples -> Archivo dentro de CsvExampleGen
- ExampleStatistics -> Archivo dentro de StatisticsGen
- Schema -> Archivo dentro de  SchemaGen, ImportSchemaGen
- ExampleAnomalies -> Archivo dentro de ExampleValidator

Observación real:

Vemos que artefactos de tipo Example también hay en la carpeta de Transform

In [48]:
for i in store.get_artifacts_by_type('Examples'):
    print(i.uri)

pipeline/CsvExampleGen/examples/1
pipeline/Transform/transformed_examples/7


A continuacións se exploran los artefactos de tipo Schema como se pide en el documento

In [49]:
for i in store.get_artifacts_by_type('Schema'):
    print(i.uri)

pipeline/SchemaGen/schema/3
pipeline/ImportSchemaGen/schema/4
pipeline/Transform/pre_transform_schema/7
pipeline/Transform/post_transform_schema/7


Obtener las propiedades de un artefacto en particular como se pide en el documento

In [50]:
for i in store.get_artifacts_by_type('Schema'):
    pp.pprint(i.custom_properties)

{'tfx_version': string_value: "1.16.0"
, 'name': string_value: "schema:2025-03-05T18:03:22.205003"
, 'producer_component': string_value: "SchemaGen"
}
{'producer_component': string_value: "ImportSchemaGen"
, 'name': string_value: "schema:2025-03-05T18:03:26.491863"
, 'tfx_version': string_value: "1.16.0"
}
{'name': string_value: "pre_transform_schema:2025-03-05T18:03:40.751800"
, 'producer_component': string_value: "Transform"
, 'tfx_version': string_value: "1.16.0"
}
{'producer_component': string_value: "Transform"
, 'name': string_value: "post_transform_schema:2025-03-05T18:03:40.751800"
, 'tfx_version': string_value: "1.16.0"
}


In [51]:
for i in store.get_artifacts_by_type('Schema'):
    pp.pprint(i.custom_properties['producer_component'])

string_value: "SchemaGen"

string_value: "ImportSchemaGen"

string_value: "Transform"

string_value: "Transform"



Se exploran las propiedades de los artefactos de tipo `ExampleStatistics` y se confirma la existencia de propiedades relacionadas con las divisiones

In [52]:
for i in store.get_artifacts_by_type('ExampleStatistics'):
    pp.pprint(i.custom_properties)

{'producer_component': string_value: "StatisticsGen"
, 'sample_rate_by_split': struct_value {
  fields {
    key: "__value__"
    value {
      string_value: "{\"eval\": 1.0, \"train\": 1.0}"
    }
  }
}
, 'name': string_value: "statistics:2025-03-05T18:03:08.540489"
, 'stats_dashboard_link': string_value: ""
, 'tfx_version': string_value: "1.16.0"
}
{'stats_dashboard_link': string_value: ""
, 'sample_rate_by_split': struct_value {
  fields {
    key: "__value__"
    value {
      string_value: "{\"eval\": 1.0, \"train\": 1.0}"
    }
  }
}
, 'producer_component': string_value: "StatisticsGen"
, 'tfx_version': string_value: "1.16.0"
, 'name': string_value: "statistics:2025-03-05T18:03:26.633369"
}
{'tfx_version': string_value: "1.16.0"
, 'producer_component': string_value: "Transform"
, 'name': string_value: "pre_transform_stats:2025-03-05T18:03:40.751800"
}
{'name': string_value: "post_transform_stats:2025-03-05T18:03:40.751800"
, 'producer_component': string_value: "Transform"
, 'tfx_

### **5.2** Seguimiento de artefactos

Se asume que no se quiere una función que liste los artefactos definiendo un tipo sino instancias usadas en el código como `SchemaGen`, `CsvExampleGen`, `Transform`. Se explica este supuesto ya que `TransfromGraph` es un tipo de artefacto y ya exploramos la función que provee el `MetadataStore` para tener los artefactos por tipo

In [53]:
def get_artifacts_by_instance(store_obj: mlmd.metadata_store.metadata_store.MetadataStore, instance: str):
    return [i for i in store.get_artifacts() if i.custom_properties['producer_component'].string_value == instance]
        

In [54]:
get_artifacts_by_instance(store, 'SchemaGen')

[id: 3
 type_id: 18
 uri: "pipeline/SchemaGen/schema/3"
 custom_properties {
   key: "name"
   value {
     string_value: "schema:2025-03-05T18:03:22.205003"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "SchemaGen"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.16.0"
   }
 }
 state: LIVE
 name: "schema:2025-03-05T18:03:22.205003"
 type: "Schema"
 create_time_since_epoch: 1741215802261
 last_update_time_since_epoch: 1741215802298]

In [55]:
get_artifacts_by_instance(store, 'Transform')

[id: 7
 type_id: 23
 uri: "pipeline/Transform/transform_graph/7"
 custom_properties {
   key: "name"
   value {
     string_value: "transform_graph:2025-03-05T18:03:40.751800"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Transform"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.16.0"
   }
 }
 state: LIVE
 name: "transform_graph:2025-03-05T18:03:40.751800"
 type: "TransformGraph"
 create_time_since_epoch: 1741215821370
 last_update_time_since_epoch: 1741215870194,
 id: 8
 type_id: 14
 uri: "pipeline/Transform/transformed_examples/7"
 properties {
   key: "split_names"
   value {
     string_value: "[\"train\", \"eval\"]"
   }
 }
 custom_properties {
   key: "name"
   value {
     string_value: "transformed_examples:2025-03-05T18:03:40.751800"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Transform"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     

### **5.3** Obtener artefactos principales

Se exploran los artefactos de tipo schema para obtener el SchemaGen. Se puede observar que este tiene un id 3, y se podría obtener los eventos relacionados

In [56]:
for i in store.get_artifacts_by_type('Schema'):
    print(i.uri, i.id, sep=" - ")

pipeline/SchemaGen/schema/3 - 3
pipeline/ImportSchemaGen/schema/4 - 4
pipeline/Transform/pre_transform_schema/7 - 10
pipeline/Transform/post_transform_schema/7 - 12


In [57]:
store.get_events_by_artifact_ids([3])

[artifact_id: 3
 execution_id: 3
 path {
   steps {
     key: "schema"
   }
   steps {
     index: 0
   }
 }
 type: OUTPUT
 milliseconds_since_epoch: 1741215802263]

Se puede observar que el `execution_id` es 3, y se pueden extraer los eventos relacionados a este id de ejecución

In [58]:
store.get_events_by_execution_ids([3])

[artifact_id: 2
 execution_id: 3
 path {
   steps {
     key: "statistics"
   }
   steps {
     index: 0
   }
 }
 type: INPUT
 milliseconds_since_epoch: 1741215802238,
 artifact_id: 3
 execution_id: 3
 path {
   steps {
     key: "schema"
   }
   steps {
     index: 0
   }
 }
 type: OUTPUT
 milliseconds_since_epoch: 1741215802263]

Aquí vemos que el evento se relaciona precisamente con el artefacto de `Statistics`, donde se indica un tipo `INPUT`, lo cual debería ser intuitivo, ya que este artefacto de tipo `InferSchemaGen` requiere de estadísticas previas. Además, observamos que finaliza en un artefacto, el cual es el `OUTPUT`.

Si exploramos el segundo schema, se observan más eventos asociados, esto es coherente debido a que este segundo se uso también en el proceso de transformación por lo que estará asociado a estos procesos

In [59]:
store.get_events_by_artifact_ids([4])

[artifact_id: 4
 execution_id: 4
 path {
   steps {
     key: "schema"
   }
   steps {
     index: 0
   }
 }
 type: OUTPUT
 milliseconds_since_epoch: 1741215806545,
 artifact_id: 4
 execution_id: 5
 path {
   steps {
     key: "schema"
   }
   steps {
     index: 0
   }
 }
 type: INPUT
 milliseconds_since_epoch: 1741215806662,
 artifact_id: 4
 execution_id: 6
 path {
   steps {
     key: "schema"
   }
   steps {
     index: 0
   }
 }
 type: INPUT
 milliseconds_since_epoch: 1741215820524,
 artifact_id: 4
 execution_id: 7
 path {
   steps {
     key: "schema"
   }
   steps {
     index: 0
   }
 }
 type: INPUT
 milliseconds_since_epoch: 1741215821334]

Entre esos eventos debe estar el que se relaciona con la generación de estadísticas realizadas después de importar el esquema. Se identifica que este está relacionado con el ID de ejecución 5. Así que al explorar los eventos, vemos que se usa el artefacto de `Example` y `ImportSchema` como entradas y da salida un artefacto de `Statistics`

In [60]:
for i in store.get_events_by_execution_ids([5]):
    print(f"ID: {i.artifact_id} - Key: {i.path.steps[0]}")

ID: 1 - Key: key: "examples"

ID: 4 - Key: key: "schema"

ID: 5 - Key: key: "statistics"



Se explora el `CsvExampleGen` mencionado en el documento

In [61]:
store.get_events_by_artifact_ids([1])

[artifact_id: 1
 execution_id: 1
 path {
   steps {
     key: "examples"
   }
   steps {
     index: 0
   }
 }
 type: OUTPUT
 milliseconds_since_epoch: 1741215758264,
 artifact_id: 1
 execution_id: 2
 path {
   steps {
     key: "examples"
   }
   steps {
     index: 0
   }
 }
 type: INPUT
 milliseconds_since_epoch: 1741215788577,
 artifact_id: 1
 execution_id: 5
 path {
   steps {
     key: "examples"
   }
   steps {
     index: 0
   }
 }
 type: INPUT
 milliseconds_since_epoch: 1741215806661,
 artifact_id: 1
 execution_id: 7
 path {
   steps {
     key: "examples"
   }
   steps {
     index: 0
   }
 }
 type: INPUT
 milliseconds_since_epoch: 1741215821334]

Se observa que se usa en 3 otros procesos como `INPUT`. Posiblemente para la primera y segunda generación de estadísticas, y en el proceso de transformación.

In [62]:
for i in store.get_events_by_execution_ids([2]):
    print(f"ID: {i.artifact_id} - Key: {i.path.steps[0]}")

ID: 1 - Key: key: "examples"

ID: 2 - Key: key: "statistics"



In [63]:
for i in store.get_events_by_execution_ids([5]):
    print(f"ID: {i.artifact_id} - Key: {i.path.steps[0]}")

ID: 1 - Key: key: "examples"

ID: 4 - Key: key: "schema"

ID: 5 - Key: key: "statistics"



In [64]:
for i in store.get_events_by_execution_ids([7]):
    print(f"ID: {i.artifact_id} - Key: {i.path.steps[0]}")

ID: 1 - Key: key: "examples"

ID: 4 - Key: key: "schema"

ID: 7 - Key: key: "transform_graph"

ID: 8 - Key: key: "transformed_examples"

ID: 9 - Key: key: "updated_analyzer_cache"

ID: 10 - Key: key: "pre_transform_schema"

ID: 11 - Key: key: "pre_transform_stats"

ID: 12 - Key: key: "post_transform_schema"

ID: 13 - Key: key: "post_transform_stats"

ID: 14 - Key: key: "post_transform_anomalies"



Se confirma la hipótesis